In [237]:
import pandas as pd

# Tratamento dos dados e criação de algumas features

In [238]:
df_estoques = pd.read_csv('../data/tb_estoques.csv')
df_estoques.head()

,key_loja,cod_produto,dt_estoque,qt_estoque
0,1,1428,2024-03-18,108
1,1,1428,2024-12-13,24
2,1,1428,2024-07-30,33
3,1,1428,2024-07-22,34
4,1,1428,2024-07-29,34


In [239]:
df_estoques.isnull().sum()

key_loja       0
cod_produto    0
dt_estoque     0
qt_estoque     0
dtype: int64

In [240]:
df_estoques.dtypes

key_loja        int64
cod_produto     int64
dt_estoque     object
qt_estoque      int64
dtype: object

In [241]:
df_estoques['dt_estoque'] = pd.to_datetime(df_estoques['dt_estoque'])
df_estoques.dtypes

key_loja                int64
cod_produto             int64
dt_estoque     datetime64[ns]
qt_estoque              int64
dtype: object

In [242]:
df_estoques = df_estoques.sort_values(['key_loja','cod_produto','dt_estoque'])
df_estoques['ruptura'] = df_estoques['qt_estoque'].apply(lambda x: 1 if x==0 else 0)
df_estoques.head()

,key_loja,cod_produto,dt_estoque,qt_estoque,ruptura
46,1,1428,2024-01-01,108,0
114,1,1428,2024-01-02,108,0
9,1,1428,2024-01-03,108,0
358,1,1428,2024-01-04,108,0
266,1,1428,2024-01-05,108,0


In [243]:
df_similares = pd.read_csv('../data/tb_similares.csv')
df_similares.head()

,cod_material,cod_material_recom,ranking
0,48635,25472,1
1,50651,74932,1
2,48062,77684,1
3,49893,48730,1
4,49943,84203,1


In [244]:
df_similares.isnull().sum()

cod_material          0
cod_material_recom    0
ranking               0
dtype: int64

In [245]:
df_similares.describe()

,cod_material,cod_material_recom,ranking
count,110.000000,110.000000,110.000000
mean,49808.772727,56650.909091,1.927273
std,15570.990743,20196.551436,0.809452
min,1428.000000,1428.000000,1.000000
25%,48562.000000,48672.250000,1.000000
50%,49881.000000,49945.000000,2.000000
75%,51349.500000,76665.500000,3.000000
max,84068.000000,93024.000000,3.000000


In [246]:
df_similares.dtypes

cod_material          int64
cod_material_recom    int64
ranking               int64
dtype: object

In [247]:
count_ranking_1 = df_similares[df_similares['ranking'] == 1].groupby('cod_material').size()

cod_material_com_mais_de_um_ranking_1 = count_ranking_1[count_ranking_1 > 1].index

df_mais_1_similar = df_similares[df_similares['cod_material'].isin(cod_material_com_mais_de_um_ranking_1)]

df_mais_1_similar

,cod_material,cod_material_recom,ranking


In [248]:
df_vendas = pd.read_csv('../data/tb_vendas.csv')
df_vendas.head()

,dt_venda,cod_pedido,canal,des_estado_franquia,key_loja,cod_material,categoria_produto,sub_categoria_produto,linha_produto,vlr_venda_tabelado,vlr_desconto
0,2024-05-10,386761455,FISCO,SP,1,1428,217.0,422944,731659,248.7,0.00
1,2024-03-25,377623716,DIGITAL,SP,1,1428,217.0,422944,731659,248.7,88.23
2,2024-03-19,376372149,DIGITAL,SP,1,1428,217.0,422944,731659,248.7,88.23
3,2024-10-23,414882456,DIGITAL,SP,1,1428,217.0,422944,731659,248.7,0.00
4,2024-02-27,371457908,DIGITAL,SP,1,1428,217.0,422944,731659,230.7,0.00


In [249]:
df_vendas.isnull().sum()

dt_venda                   0
cod_pedido                 0
canal                      0
des_estado_franquia        0
key_loja                   0
cod_material               0
categoria_produto         34
sub_categoria_produto      0
linha_produto              0
vlr_venda_tabelado         0
vlr_desconto             270
dtype: int64

In [250]:
df_vendas.dtypes

dt_venda                  object
cod_pedido                 int64
canal                     object
des_estado_franquia       object
key_loja                   int64
cod_material               int64
categoria_produto        float64
sub_categoria_produto      int64
linha_produto              int64
vlr_venda_tabelado       float64
vlr_desconto             float64
dtype: object

In [251]:
df_vendas['dt_venda'] = pd.to_datetime(df_vendas['dt_venda'])
df_vendas['categoria_produto'] = df_vendas['categoria_produto'].fillna(0).astype(int)
df_vendas['vlr_desconto'] = df_vendas['vlr_desconto'].fillna(0)
df_vendas.dtypes

dt_venda                 datetime64[ns]
cod_pedido                        int64
canal                            object
des_estado_franquia              object
key_loja                          int64
cod_material                      int64
categoria_produto                 int32
sub_categoria_produto             int64
linha_produto                     int64
vlr_venda_tabelado              float64
vlr_desconto                    float64
dtype: object

In [252]:
df_vendas.isnull().sum()

dt_venda                 0
cod_pedido               0
canal                    0
des_estado_franquia      0
key_loja                 0
cod_material             0
categoria_produto        0
sub_categoria_produto    0
linha_produto            0
vlr_venda_tabelado       0
vlr_desconto             0
dtype: int64

# 1.  Análise do grau de substituição dos SKUs em ruptura pelos itens similares recomendados pelo modelo.

In [253]:
df_estoques_ruptura = df_estoques.copy()
df_estoques_ruptura = df_estoques_ruptura[df_estoques_ruptura['ruptura'] == 1]
df_estoques_ruptura.head()

,key_loja,cod_produto,dt_estoque,qt_estoque,ruptura
92,1,1428,2024-11-19,0,1
243,1,1428,2024-11-20,0,1
323,1,1428,2024-11-21,0,1
247,1,1428,2024-11-22,0,1
1217,1,22230,2024-01-01,0,1


In [254]:
df_similares = df_similares.sort_values(['cod_material','ranking'])
df_similares = df_similares[df_similares['ranking']==1]
df_similares = df_similares.rename(columns={'cod_material': 'cod_produto', 'cod_material_recom': 'cod_produto_recom'})
df_similares.head()

,cod_produto,cod_produto_recom,ranking
32,1428,1650,1
30,1650,1428,1
37,25458,75610,1
5,47851,47881,1
15,47905,80876,1


In [255]:
df_estoques_ruptura = df_estoques_ruptura.merge(df_similares, on='cod_produto', how='left')
numero_de_linhas = len(df_estoques_ruptura)
print(f'O DataFrame tem {numero_de_linhas} linhas.')

O DataFrame tem 91504 linhas.


In [256]:
df_estoques_ruptura = df_estoques_ruptura[df_estoques_ruptura['cod_produto_recom'].notna()]
numero_de_linhas = len(df_estoques_ruptura)
print(f'O DataFrame tem {numero_de_linhas} linhas.')

O DataFrame tem 22368 linhas.


In [257]:
df_estoques_ruptura.head()

,key_loja,cod_produto,dt_estoque,qt_estoque,ruptura,cod_produto_recom,ranking
0,1,1428,2024-11-19,0,1,1650.0,1.0
1,1,1428,2024-11-20,0,1,1650.0,1.0
2,1,1428,2024-11-21,0,1,1650.0,1.0
3,1,1428,2024-11-22,0,1,1650.0,1.0
1144,1,47851,2024-11-29,0,1,47881.0,1.0


In [258]:
df_estoques_ruptura = df_estoques_ruptura.reset_index()

In [259]:
df_vendas.head()

,dt_venda,cod_pedido,canal,des_estado_franquia,key_loja,cod_material,categoria_produto,sub_categoria_produto,linha_produto,vlr_venda_tabelado,vlr_desconto
0,2024-05-10,386761455,FISCO,SP,1,1428,217,422944,731659,248.7,0.00
1,2024-03-25,377623716,DIGITAL,SP,1,1428,217,422944,731659,248.7,88.23
2,2024-03-19,376372149,DIGITAL,SP,1,1428,217,422944,731659,248.7,88.23
3,2024-10-23,414882456,DIGITAL,SP,1,1428,217,422944,731659,248.7,0.00
4,2024-02-27,371457908,DIGITAL,SP,1,1428,217,422944,731659,230.7,0.00


In [280]:
df_vendas_select = df_vendas.sort_values(['key_loja','dt_venda','cod_material']).copy()
df_vendas_select_2 = df_vendas.sort_values(['key_loja','dt_venda','cod_material']).copy()

df_vendas_select = df_vendas_select[['key_loja','dt_venda','cod_material','vlr_venda_tabelado','vlr_desconto']]
df_vendas_agg = df_vendas_select.groupby(['key_loja','dt_venda','cod_material']).agg(qt_total_vendido=('cod_material', 'count')).reset_index()
# df_vendas_agg = df_vendas_select.groupby(['key_loja','dt_venda','cod_material']).agg(vlr_venda_total=('vlr_venda_tabelado', 'sum'), total_vendido=('cod_material', 'count')).reset_index()


# df_vendas_select_2 = df_vendas_select_2[['key_loja','dt_venda','cod_material','vlr_venda_tabelado','vlr_desconto']]
# df_vendas_agg_2 = df_vendas_select_2.groupby(['key_loja','dt_venda','cod_material']).agg(vlr_venda_tabelado=('vlr_venda_tabelado', 'max'), vlr_desconto=('vlr_desconto', 'max')).reset_index()

# df_vendas_agg = df_vendas_agg.merge(df_vendas_agg_2, on=['key_loja','dt_venda','cod_material'], how='left')
df_vendas_agg = df_vendas_agg.rename(columns={'cod_material': 'cod_produto'})

df_vendas_agg.head()

,key_loja,dt_venda,cod_produto,qt_total_vendido
0,1,2024-01-02,48062,1
1,1,2024-01-02,48635,1
2,1,2024-01-02,82060,1
3,1,2024-01-03,48062,1
4,1,2024-01-03,48253,1


In [293]:
df_vendas_agg_10 = df_vendas_agg[df_vendas_agg['dt_venda']=='2024-12-27']
df_vendas_agg_10 = df_vendas_agg_10[df_vendas_agg_10['key_loja']==1]
df_vendas_agg_10 = df_vendas_agg_10[df_vendas_agg_10['cod_produto']==85124]

df_vendas_agg_10.head()

,key_loja,dt_venda,cod_produto,qt_total_vendido


In [294]:
df_estoques_ruptura_10 = df_estoques_ruptura[df_estoques_ruptura['dt_estoque']=='2024-10-31']
df_estoques_ruptura_10 = df_estoques_ruptura_10[df_estoques_ruptura_10['key_loja']==1]
df_estoques_ruptura_10 = df_estoques_ruptura_10[df_estoques_ruptura_10['cod_produto']==48145]

df_estoques_ruptura_10.head()

,index,key_loja,cod_produto,dt_estoque,qt_estoque,ruptura,cod_produto_recom,ranking
40,1373,1,48145,2024-10-31,0,1,77241.0,1.0


In [298]:
test = df_estoques_ruptura.copy()
test['cod_produto_recom'] = test['cod_produto_recom'].astype(int)
test = test.rename(columns={'dt_estoque': 'dt'})\
           .merge(df_vendas_agg.rename(columns={'dt_venda': 'dt', 'cod_produto': 'cod_produto_recom'}), on=['key_loja','cod_produto_recom','dt'], how='left')

test['qt_total_vendido'] = test['qt_total_vendido'].fillna(0)


# test2 = test[test['total_vendido'].notna()]
# test2

substituicao = test.groupby(['key_loja','cod_produto_recom','dt'])['qt_total_vendido'].sum().reset_index()

# Exibindo os resultados
substituicao

,key_loja,cod_produto_recom,dt,qt_total_vendido
0,1,1650,2024-11-19,0.0
1,1,1650,2024-11-20,0.0
2,1,1650,2024-11-21,0.0
3,1,1650,2024-11-22,0.0
4,1,24499,2024-01-01,0.0
...,...,...,...,...
20443,8,85124,2024-12-27,0.0
20444,8,85124,2024-12-28,0.0
20445,8,85124,2024-12-29,0.0
20446,8,85124,2024-12-30,0.0
